In [1]:
import pandas as pd

In [2]:
import numpy as np

In [3]:
import matplotlib.pyplot as plt

In [4]:
from random import choice

In [5]:
from random import shuffle

In [6]:
import csv

In [7]:
villCentroid = pd.read_csv('SelectedStatesVillagesCentroid.csv')

In [8]:
x_min = min(villCentroid['CentX'])
x_max = max(villCentroid['CentX'])
y_min = min(villCentroid['CentY'])
y_max = max(villCentroid['CentY'])

In [9]:
n_r = 75
n_c = 75

In [10]:
x_range = x_max-x_min
y_range = y_max-y_min

In [11]:
### index to fit will be (col*n_r + row) if the 2d matrix is reperesented in 1d list form

In [12]:
## creating new id which is unique for all villages 
villCentroid['UID'] = range(len(villCentroid))

In [13]:
villCentroid.to_csv('villCentroidUID.csv')

In [14]:
cell_length = x_range/((n_c-1)*1.0)
cell_height = y_range/((n_r-1)*1.0)

sorted_villList =  [[] for i in range(n_c*n_r)]
vill_gridNo = [0]*len(villCentroid)
for i in range(len(villCentroid)):
    y = villCentroid['CentY'][i:i+1].values[0].astype(float)
    x = villCentroid['CentX'][i:i+1].values[0].astype(float)
    #vill_code11 = villCentroid['village_code_2011'][i:i+1].values[0].astype(float)
    vill_codeUID = villCentroid['UID'][i:i+1].values[0].astype(float)
    
    c = int((x-x_min)/cell_length)
    r = int((y-y_min)/cell_height)
    
    idx = c*n_r + r
    vill_gridNo[i] = idx
    sorted_villList[idx].append(vill_codeUID)

In [15]:
## now lets work on splitting it up in Train:CV:Test
train= 50
cv = 30
test = 20

In [16]:
sparse_vills = []
Train_vills = []
CV_vills = []
Test_vills = []
for vill_list in sorted_villList:
    if len(vill_list) < 10:
        sparse_vills.extend(vill_list)
        continue
        
    shuffle(vill_list)
    lenTrain = int(round(train/100.0*len(vill_list)))
    lenCV = int(round(cv/100.0*len(vill_list)))
    Train_vills.extend(vill_list[:lenTrain])
    CV_vills.extend(vill_list[lenTrain:lenTrain+lenCV])
    Test_vills.extend(vill_list[lenTrain+lenCV:])

## now splitting the sparse_vills list and adding it to train,cv and test sets
lenTrain = int(round(train/100.0*len(sparse_vills)))
lenCV = int(round(cv/100.0*len(sparse_vills)))
Train_vills.extend(sparse_vills[:lenTrain])
CV_vills.extend(sparse_vills[lenTrain:lenTrain+lenCV])
Test_vills.extend(sparse_vills[lenTrain+lenCV:])

In [17]:
df_trainVills = pd.DataFrame(Train_vills)
df_trainVills.to_csv('Train_vills.csv','w',index=False,header=False)

df_cvVills = pd.DataFrame(CV_vills)
df_cvVills.to_csv('CV_vills.csv','w',index=False,header=False)

df_testVills =pd.DataFrame(Test_vills)
df_trainVills.to_csv('Test_vills.csv','w',index=False,header=False)

In [18]:
del villCentroid['Unnamed: 0']

In [19]:
vill = pd.read_csv('Vill.csv')

In [20]:
villCentroid.head(3)

,ID,village_code_2011,village_code_2001,CentX,CentY,UID
0,BR-0,215990,200.0,83.923688,27.421459,0
1,BR-1,215989,100.0,83.937193,27.437632,1
2,BR-2,215991,300.0,83.875222,27.417117,2


In [21]:
df_vill = vill.merge(villCentroid[['UID','village_code_2011']],left_on='Town/Village',right_on='village_code_2011',how='inner')

In [22]:
del df_vill['Unnamed: 0']
del df_vill['Unnamed: 0.1']

In [23]:
df_vill.to_csv('Vill_Uid.csv')

In [24]:
df_trainVills['set_type'] = ['train' for i in range(len(df_trainVills))]

In [25]:
df_testVills['set_type'] = ['test' for i in range(len(df_testVills))]

In [26]:
df_cvVills['set_type'] = ['CV' for i in range(len(df_cvVills))]

In [27]:
df_allSets = pd.concat([df_trainVills,df_cvVills,df_testVills])

In [28]:
df_allSets.columns = ['UID','set_type']

In [29]:
df_vill = df_vill.merge(df_allSets,left_on='UID',right_on='UID',how='inner')

In [30]:
del df_vill['Town/Village']

In [31]:
df_vill.to_csv('vill_setType.csv')

In [32]:
df_vill.head(3)

,State,District,Subdistt,Ward,EB,Level,Name,TRU,ELG_POP,No_HH,...,District_HHD_Cluster_CHH,Village_HHD_Cluster_FC,District_HHD_Cluster_FC,Village_HHD_Cluster_BF,District_HHD_Cluster_BF,Village_HHD_Cluster_EMP,District_HHD_Cluster_EMP,UID,village_code_2011,set_type
0,10,203,1013,0,0,VILLAGE,Bhaisalotan,Rural,9046,1954,...,3. Developed,1. Under-Developed,2. Moderately-Developed,1. Under-Developed,1. Under-Developed,1. High Unemployement,1. High Unemployement,0,215990,train
1,10,203,1013,0,0,VILLAGE,Tharhi,Rural,2174,482,...,3. Developed,1. Under-Developed,2. Moderately-Developed,1. Under-Developed,1. Under-Developed,2. High AL,1. High Unemployement,2,215991,test
2,10,203,1013,0,0,VILLAGE,Pipra,Rural,1459,319,...,3. Developed,1. Under-Developed,2. Moderately-Developed,1. Under-Developed,1. Under-Developed,2. High AL,1. High Unemployement,5,215992,test


In [33]:
vill_labels = ['Village_HHD_Cluster_MSL',
'Village_HHD_Cluster_MSW',
'Village_HHD_Cluster_CHH',
'Village_HHD_Cluster_FC',
'Village_HHD_Cluster_BF',
'Village_HHD_Cluster_EMP']

In [34]:
def percConvert(ser):
    return ser/float(ser[-1])

In [35]:
df_split_dist = pd.DataFrame()
for label in vill_labels:
    print (pd.crosstab(df_vill['set_type'],df_vill[label],margins=True).apply(percConvert, axis=1))

Village_HHD_Cluster_MSL  1. Under-Developed  2. Moderately-Developed  \
set_type                                                               
CV                                 0.347836                 0.216024   
test                               0.349139                 0.218714   
train                              0.345755                 0.218360   
All                                0.347050                 0.217728   

Village_HHD_Cluster_MSL  3. Developed  All  
set_type                                    
CV                           0.436139  1.0  
test                         0.432147  1.0  
train                        0.435885  1.0  
All                          0.435222  1.0  
Village_HHD_Cluster_MSW  1. Under-Developed  2. Moderately-Developed  \
set_type                                                               
CV                                 0.208821                 0.502217   
test                               0.206089                 0.506408   
train    

In [38]:
df =pd.crosstab(df_vill['set_type'],df_vill[label],margins=True).apply(percConvert, axis=1)

In [40]:
pd.pivot_table(df,df)

ValueError: No group keys passed!

In [ ]:
df_vill.head()
del df_vill['count']

In [ ]:
df_vill.head()

In [ ]:
vill_train = df[df_vill['set_type'] == 'train']
vill_train.to_csv('vill_train.csv')

In [ ]:
vill_test = df[df_vill['set_type'] == 'test']
vill_test.to_csv('vill_test.csv')

In [ ]:
vill_cv = df[df_vill['set_type'] == 'CV']
vill_cv.to_csv('vill_cv.csv')